In [36]:
import numpy as np
import pandas as pd
import re
import jieba

In [37]:
data0=pd.read_excel("./data/dataset/dataset0.xls")
data1=data0.sample(frac=1.0)
y=data1.tcms_code
outm=data1.iloc[:,8:]
context=data1.contxt
diag=data1.diag
data1.head()

,hid,id,pid,contxt,diag,diag_code,tcm_syn,tcms_code,tan,feng,han,ren,qixu,yinxu,fei,pi,shen
13224,2,13225,201914264,反复咳嗽咳痰，活动后气促10余年，加重1周。 望之有神、表情正常，面色荣润，形体适中；行动自...,支气管扩张症伴感染,6,痰浊阻肺,9,1,0,0,0,0,0,1,0,0
2797,1,2798,1610662,右肺癌伽马刀治疗后一年半余，肢体活动障碍三月余 患者面色憔悴，色稍黑；咳喘，无喉中水鸡声；胸...,肺癌,0,肺阴虚,2,0,0,0,0,0,1,1,0,0
7988,1,7989,1711338,咳嗽咳痰两周，恶寒发热四天。 :该患者以咳嗽、咯痰、发热为主诉，结合患者舌脉，舌质红，苔黄...,肺部感染,1,痰热壅肺,8,1,0,0,1,0,0,1,0,0
13451,2,13452,201929702,反复咳嗽咳痰、气促20年，再发加重3天 望之少神、面色少华，形体适中；行动自如、精神不振、发...,慢性阻塞性肺病伴感染,5,痰浊阻肺,9,1,0,0,0,0,0,1,0,0
8918,1,8919,1904255,间断右侧胸痛1月。 患者右侧胸痛，与呼吸运动相关，偶有咳嗽，仅少量白痰，饮食及睡眠正常，二便...,肺部感染,1,痰热壅肺,8,1,0,0,1,0,0,1,0,0


In [38]:
print(data1.shape)
print(y.shape)
print(outm.shape)
print(context.shape)
print(diag.shape)
print(set(data1.diag_code))

(14075, 17)
(14075,)
(14075, 9)
(14075,)
(14075,)
{0, 1, 2, 3, 4, 5, 6, 7}


In [39]:
data0_ex=pd.read_excel("./data/dataset/external0.xls")
data_ex=data0_ex.sample(frac=1.0)
y_ex=data_ex.tcms_code
outm_ex=data_ex.iloc[:,8:]
context_ex=data_ex.contxt
diag_ex=data_ex.diag
data_ex.head()

,hid,id,pid,contxt,diag,diag_code,tcm_syn,tcms_code,tan,feng,han,ren,qixu,yinxu,fei,pi,shen
508,1,5489,1501490,反复发作性喘息胸闷六十余年，再发一周。 神志清，稍气喘貌，喘息，胸闷，喉中哮鸣有声，咳嗽，咳...,支气管哮喘,7,寒哮,5,0,0,1,0,0,0,1,0,0
83,1,231,1505037,右肺癌末次化疗后2月，脑转移放疗后1月。 患者面色稍黑；无咳喘，无喉中水鸡声；胸廓呈正常；言...,肺癌,0,肺脾气虚,0,0,0,0,0,1,0,1,1,0
290,2,2263,201931508,间断咳嗽10余年，加重1周。 望之少神、表情正常，面色荣润，形体适中；行动自如、精神良好、发...,慢性支气管炎,4,肺阴虚,2,0,0,0,0,0,1,1,0,0
363,1,3294,1609934,反复咳嗽咳痰近一年，加重伴胸闷气喘两月余。 阵发性咳嗽，咳少量白粘痰，尚易咯出，胸闷气喘，活...,间质性肺炎伴感染,3,风寒袭肺,3,0,1,1,0,0,0,1,0,0
990,2,12982,201924257,反复咳嗽、咳痰50余年，再发加重1周。 望之有神、表情正常，面色荣润，形体适中；行动自如、精...,支气管扩张症伴感染,6,痰浊阻肺,9,1,0,0,0,0,0,1,0,0


In [40]:
print(data_ex.shape)
print(y_ex.shape)
print(outm_ex.shape)
print(context_ex.shape)
print(diag_ex.shape)
print(set(data_ex.diag_code))

(1000, 17)
(1000,)
(1000, 9)
(1000,)
(1000,)
{0, 1, 2, 3, 4, 5, 6, 7}


In [41]:
dictfile="./data/tcm_all_dict_2.txt"
stopwords="./data/stopwords.txt"
jieba.load_userdict(dictfile)   

In [42]:
def file2doc1(infile,dictfile):    
    jieba.load_userdict(dictfile)                     
    doc = [w for x in codecs.open(file_name, 'r', 'utf-8').readlines() for w in jieba.cut(x.strip())]
    return doc

In [43]:
def load_stopword(filename):  
    stopwords = [line.strip() for line in open(filename, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [44]:
def extract_chn(data):
    pattern="[\u4e00-\u9fa5]"
    regex=re.compile(pattern)
    res=regex.findall(data)
    #return "".join(res)
    return res

In [45]:
def drop_alnum(astr):
    drop_str="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    str1=astr
    for ch in str1:
        if ch in drop_str:
            str1=str1.replace(ch,"")
    return str1

In [46]:
def get_corpus(context):
    corpus=[]    
    
    for sent in context:
        sent=sent+"；"
        sent=del_neg_txt1(sent) #drop negative words  
        sent=del_neg_txt2(sent)
        sent.strip("；")
        strlist=list(jieba.cut(sent))
        strlist=sorted(set(strlist),key=strlist.index)
        corpus.append(strlist)
        
    return corpus

In [47]:
def get_corpus1(doc,stopwords):    
    stopword_list=load_stopword(stopwords)   
    #r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'   #drop letters, numbers and sepcial chars
    out=[]
    
    for sent in doc:        
        doc_stopword=[y for y in sent if y not in stopword_list] #drop stopword
        res=[]
        for y in doc_stopword:
            #res.append(re.sub(r1,"",y))   #drop letters, numbers and sepcial chars
            y=drop_alnum(y)
            res.append(y)  #drop letters, numbers and sepcial chars
        out.append(res)
    return out

In [48]:
def get_corpus2(corpus,diagnosis):        
    out=list()  
    
    LEN=len(corpus)
    diaglist=list(diagnosis)
    for i in range(LEN):
        inlist=list()
        inlist.extend(corpus[i])        
        inlist.append(diaglist[i])
        out.append(inlist)        
    return out

In [49]:
def del_neg_txt1(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="无"   
    item1=["；","，","舌"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [50]:
def del_neg_txt2(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="未"
    item1=["；","，","有"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [51]:
atxt=get_corpus(context)
#print(atxt)
cps=get_corpus1(atxt, stopwords)
#print(len(cps))
cps1=get_corpus2(cps,diag)
#i=0
#for sent in cps1:
    #print(i, sent)
    #i+=1  

In [52]:
print(len(cps))

14075


In [56]:
atxt_ex=get_corpus(context_ex)
#print(atxt)
cps_ex=get_corpus1(atxt_ex, stopwords)
#print(len(cps))
cps1_ex=get_corpus2(cps,diag)

In [57]:
print(len(cps_ex))

1000


In [58]:
import gensim
import codecs
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [59]:
import time

st1=time.time()
#model_path=("./model/mode821.d2v")
#documents=[TaggedDocument(doc,[i]) for i, doc in enumerate(cps)] #tagged corpus
#model=Doc2Vec(documents,vector_size=512,window=5,min_count=5,workers=8)
#model.save(model_path)
#model_d2v=Doc2Vec.load(model_path)

model_path=("./model_d2v/zhiwiki_news.doc2vec")
model_d2v=Doc2Vec.load(model_path)

st2=time.time()
print("elapse: %f minutes"%((st2-st1)/60.0))

d:\code\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


elapse: 0.063813 minutes


d:\code\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [60]:
import time

In [61]:
def get_vectors(corpus,model_d2v):
    vectors=[]
    LEN=len(corpus)   
    
    step=100
    steplen=int(LEN/step)+1
    print("steplen = %s "%(steplen))
    
    i=0    
    st1=time.time()
    
    for sent in corpus:
        if i%steplen==0:
            st2=time.time()            
            print("complete: %s / %s, elapse: %.3f minutes."%(i/steplen,step,(st2-st1)/60.0))           
            
        vectors.append(model_d2v.infer_vector(sent))
        i+=1        
        
    return vectors

In [62]:
vec_ex_d2v=get_vectors(cps_ex,model_d2v)

steplen = 11 
complete: 0.0 / 100, elapse: 0.000 minutes.
complete: 1.0 / 100, elapse: 0.255 minutes.
complete: 2.0 / 100, elapse: 0.491 minutes.
complete: 3.0 / 100, elapse: 0.778 minutes.
complete: 4.0 / 100, elapse: 0.991 minutes.
complete: 5.0 / 100, elapse: 1.282 minutes.
complete: 6.0 / 100, elapse: 1.519 minutes.
complete: 7.0 / 100, elapse: 1.741 minutes.
complete: 8.0 / 100, elapse: 1.930 minutes.
complete: 9.0 / 100, elapse: 2.130 minutes.
complete: 10.0 / 100, elapse: 2.408 minutes.
complete: 11.0 / 100, elapse: 2.638 minutes.
complete: 12.0 / 100, elapse: 2.885 minutes.
complete: 13.0 / 100, elapse: 3.083 minutes.
complete: 14.0 / 100, elapse: 3.258 minutes.
complete: 15.0 / 100, elapse: 3.446 minutes.
complete: 16.0 / 100, elapse: 3.650 minutes.
complete: 17.0 / 100, elapse: 3.874 minutes.
complete: 18.0 / 100, elapse: 4.103 minutes.
complete: 19.0 / 100, elapse: 4.311 minutes.
complete: 20.0 / 100, elapse: 4.475 minutes.
complete: 21.0 / 100, elapse: 4.665 minutes.
comple

In [64]:
#x_ex_df=pd.DataFrame(vec_ex_d2v)
#y_ex_df=pd.DataFrame(y_ex)
#outm_ex_df=pd.DataFrame(outm_ex)
#x_ex_df.to_excel("./data/dataset/x_ex_vec_d2v.xls", header=True, index=True, index_label ="nid", encoding="utf-8")    
#y_ex_df.to_excel("./data/dataset/y_ex_vec_d2v.xls", header=True, index=True,  index_label ="nid", encoding="utf-8")   
#outm_ex_df.to_excel("./data/dataset/outm_ex_vec_d2v.xls", header=True, index=True,  index_label ="nid", encoding="utf-8")    

In [66]:
#x_ex1=pd.read_excel("./data/dataset/x_ex_vec_d2v.xls")
#y_ex1=pd.read_excel("./data/dataset/y_ex_vec_d2v.xls")
#outm_ex1=pd.read_excel("./data/dataset/outm_ex_vec_d2v.xls")

In [70]:
#x_ex1.head()

In [71]:
vec_d2v=get_vectors(cps,model_d2v)

steplen = 141 
complete: 0.0 / 100, elapse: 0.000 minutes.
complete: 1.0 / 100, elapse: 2.730 minutes.
complete: 2.0 / 100, elapse: 5.553 minutes.
complete: 3.0 / 100, elapse: 8.206 minutes.
complete: 4.0 / 100, elapse: 10.750 minutes.
complete: 5.0 / 100, elapse: 13.474 minutes.
complete: 6.0 / 100, elapse: 16.302 minutes.
complete: 7.0 / 100, elapse: 19.013 minutes.
complete: 8.0 / 100, elapse: 21.691 minutes.
complete: 9.0 / 100, elapse: 24.339 minutes.
complete: 10.0 / 100, elapse: 26.866 minutes.
complete: 11.0 / 100, elapse: 29.547 minutes.
complete: 12.0 / 100, elapse: 32.235 minutes.
complete: 13.0 / 100, elapse: 35.001 minutes.
complete: 14.0 / 100, elapse: 37.425 minutes.
complete: 15.0 / 100, elapse: 39.876 minutes.
complete: 16.0 / 100, elapse: 42.526 minutes.
complete: 17.0 / 100, elapse: 45.148 minutes.
complete: 18.0 / 100, elapse: 47.737 minutes.
complete: 19.0 / 100, elapse: 50.286 minutes.
complete: 20.0 / 100, elapse: 52.935 minutes.
complete: 21.0 / 100, elapse: 55.

In [84]:
x_df=pd.DataFrame(vec_d2v)
y_df=pd.DataFrame(y)
outm_df=pd.DataFrame(outm)
x_df.to_excel("./data/dataset/x_vec_d2v.xls", header=True, index=True, index_label ="nid", encoding="utf-8")    
y_df.to_excel("./data/dataset/y_vec_d2v.xls", header=True, index=True,  index_label ="nid", encoding="utf-8")   
outm_df.to_excel("./data/dataset/outm_vec_d2v.xls", header=True, index=True,  index_label ="nid", encoding="utf-8")    

In [166]:
x1=pd.read_excel("./data/dataset/x_vec_d2v.xls")
y1=pd.read_excel("./data/dataset/y_vec_d2v.xls")
outm1=pd.read_excel("./data/dataset/outm_vec_d2v.xls")

In [121]:
x_ex=pd.DataFrame(vec_ex_d2v)
y_ex=pd.DataFrame(data_ex.tcms_code)
ym_ex=pd.DataFrame(outm_ex)

In [124]:
ym_ex.shape

(1000, 9)

In [167]:
y1.head()

,nid,tcms_code
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [90]:
#x1=pd.read_csv("./data/vec_d2vpre.csv",index_col=0)
#x1.head()

In [155]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
x1=pd.DataFrame(vec_d2v)
y1=pd.DataFrame(data1.tcms_code)
x_train, x_test, y_train, y_test=train_test_split(xs1,ys1,test_size=0.25,random_state=0)
dts=train_test_split(x1,y1,test_size=0.25,random_state=0)

In [156]:
x1.shape

(14075, 192)

In [93]:
#x.to_csv("./data/vec_d2vpre.csv")

In [94]:
# multiple class output

In [157]:
from sklearn.ensemble import RandomForestClassifier
def test_rf(dts):
    cls = RandomForestClassifier(n_estimators=100)
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(y_predict,dts[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dts[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dts[3], average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3])) 
    
    y_pred_ex=cls.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 
    return y_predict

In [158]:
y_predict1=test_rf(dts)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


acc1: 0.8209718670076727
prec1: 0.8267232852673391
recall1: 0.8209718670076727
fscore1: 0.8189058952864022
acc_ex: 0.818
prec_ex: 0.8504603956930296
rec_ex: 0.818
f1_ex: 0.8260456798180515


In [151]:
print(len(y_predict1))

3519


In [152]:
y_pred1=pd.DataFrame(y_predict1,columns=["pred"])
#print(y_pred1)
#y_ture=list(y_test)
#print(y_test)
y_true1=dts[3].reset_index(drop=True)
#print(y_true1)

y_true_pred=pd.concat([y_true1,y_pred1],axis=1)
y_true_pred.columns=["true","pred"]

In [153]:
acc_list=list()
prec_list=list()
rec_list=list()
f1_list=list()
for i in range(1000):
    resample=y_true_pred.sample(frac=1.0,replace=True)
    #print(resample)
    y_t=resample.true
    y_p=resample.pred
    #print(y_t)
    acc_list.append(metrics.accuracy_score(y_t, y_p))
    prec_list.append(metrics.precision_score(y_t, y_p, average='weighted'))
    rec_list.append(metrics.recall_score(y_t, y_p, average='weighted'))
    f1_list.append(metrics.f1_score(y_t, y_p, average='weighted'))
    #print("prec1:", i, prec_list[i])
print(np.median(acc_list),np.percentile(acc_list,2.5), np.percentile(acc_list,97.5))    
print(np.median(prec_list),np.percentile(prec_list,2.5), np.percentile(prec_list,97.5))
print(np.median(rec_list),np.percentile(rec_list,2.5), np.percentile(rec_list,97.5))
print(np.median(f1_list),np.percentile(f1_list,2.5), np.percentile(f1_list,97.5))

0.8320545609548167 0.8198351804489912 0.844281045751634
0.8457435927363861 0.8345066495620694 0.8567554770682063
0.8320545609548167 0.8198351804489912 0.844281045751634
0.8337909082113255 0.8217970238507225 0.8458720356110483


In [159]:
from sklearn.neural_network import MLPClassifier
def test_ann(dts):
    cls=MLPClassifier(alpha=1e-5,hidden_layer_sizes=(256, 512,256,128,64,32))
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(y_predict,dts[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dts[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dts[3], average='weighted'))
    #print("acc1:",metrics.roc_auc_score(y_predict,dts[3]))
    
    y_pred_ex=cls.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 

In [160]:
 test_ann(dts)

d:\code\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc1: 0.8800795680591077
prec1: 0.8809613985738196
recall1: 0.8800795680591077
fscore1: 0.8801933370574647
acc_ex: 0.918
prec_ex: 0.9190488394899685
rec_ex: 0.918
f1_ex: 0.9182128285059472


In [130]:
from xgboost import XGBClassifier

def test_xgboost(dts):
    cls = XGBClassifier()
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(y_predict,dts[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dts[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dts[3], average='weighted'))
    
    y_pred_ex=cls.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 

In [131]:
test_xgboost(dts)

d:\code\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\code\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc1: 0.7584541062801933
prec1: 0.7682383773455269
recall1: 0.7584541062801933
fscore1: 0.7589520308189005
acc_ex: 0.727
prec_ex: 0.7735578940809711
rec_ex: 0.727
f1_ex: 0.7391730247762807


In [132]:
from sklearn import svm
def test_svc(dts):
    cls=svm.SVC(kernel="rbf")
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(y_predict,dts[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dts[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dts[3], average='weighted'))
    
    y_pred_ex=cls.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 

In [133]:
test_svc(dts)

d:\code\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\code\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


acc1: 0.4921852799090651
prec1: 0.7684852150673439
recall1: 0.4921852799090651
fscore1: 0.5748385385112214


d:\code\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
d:\code\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


acc_ex: 0.292
prec_ex: 0.5743765730425288
rec_ex: 0.292
f1_ex: 0.22888245830984866


d:\code\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
d:\code\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [161]:
from sklearn.neighbors import KNeighborsClassifier
def test_knn(dts):
    cls = KNeighborsClassifier()
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(y_predict,dts[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dts[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dts[3], average='weighted'))
    #print("acc:",metrics.accuracy_score(y_predict,dts[3]))
    
    y_pred_ex=cls.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 

In [162]:
test_knn(dts)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


acc1: 0.8527990906507531
prec1: 0.8588146415261706
recall1: 0.8527990906507531
fscore1: 0.8535706830302633
acc_ex: 0.902
prec_ex: 0.9046882567992639
rec_ex: 0.902
f1_ex: 0.9015156474781747


In [ ]:
# multiple label output

In [136]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
xm=pd.DataFrame(vec_d2v)
ym=pd.DataFrame(outm)
x_train, x_test, y_train, y_test=train_test_split(xm,ym,test_size=0.25,random_state=0)
dtsm=train_test_split(xm,ym,test_size=0.25,random_state=0)

In [137]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier

In [138]:
from sklearn.ensemble import RandomForestClassifier
def test_rfm(dtsm):
    rfm=RandomForestClassifier(n_estimators=100)
    clsm=MultiOutputClassifier(rfm)
    clsm.fit(dtsm[0],dtsm[2])
    y_predict=clsm.predict(dtsm[1])
    #y_predict0=clsm.fit(dtsm[0],dtsm[2]).predict(dtsm[0])
    #print("acc0:",metrics.accuracy_score(y_predict0,dtsm[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dtsm[3]))
    print("prec1:",metrics.precision_score(y_predict,dtsm[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dtsm[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dtsm[3], average='weighted'))
    
    y_pred_ex=clsm.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))  
    #print(y_predict) 

In [139]:
test_rfm(dtsm)

acc1: 0.638533674339301
prec1: 0.857544892955902
recall1: 0.9737363072456117
fscore1: 0.9044758640122867
acc_ex: 0.594
prec_ex: 0.9728775658029615
rec_ex: 0.8001537279016141
f1_ex: 0.864197020415248


In [140]:
from sklearn.neural_network import MLPClassifier
def test_mlpm(dtsm):      
    mlp=MLPClassifier(alpha=1e-5,hidden_layer_sizes=(512,1024,512,256,128,64),random_state=0)
    clsm=MultiOutputClassifier(mlp)
    clsm.fit(dtsm[0],dtsm[2])
    #y_predict=clsm.fit(dtsm[0],dtsm[1]).predict(dtsm[2])
    y_predict=clsm.predict(dtsm[1])    
    print("acc1:",metrics.accuracy_score(y_predict,dtsm[3]))
    print("prec1:",metrics.precision_score(y_predict,dtsm[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dtsm[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dtsm[3], average='weighted'))
    #print("acc:",metrics.accuracy_score(y_predict,dtsm[3]))
    #print(y_predict)
    
    y_pred_ex=clsm.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))  

In [141]:
test_mlpm(dtsm)

acc1: 0.8189826655299801
prec1: 0.9550761542980194
recall1: 0.9507470946319867
fscore1: 0.95249460625039
acc_ex: 0.843
prec_ex: 0.967895913530686
rec_ex: 0.9646425826287471
f1_ex: 0.9644777497561905


In [142]:
from xgboost import XGBClassifier
def test_xgboost(dtsm):
    xgbm = XGBClassifier()
    clsm=MultiOutputClassifier(xgbm)
    clsm.fit(dtsm[0],dtsm[2])
    #y_predict=clsm.fit(dtsm[0],dtsm[1]).predict(dtsm[2])
    y_predict=clsm.predict(dtsm[1])    
    print("acc1:",metrics.accuracy_score(y_predict,dtsm[3]))
    print("prec1:",metrics.precision_score(y_predict,dtsm[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dtsm[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dtsm[3], average='weighted'))
    
    y_pred_ex=clsm.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted')) 

In [143]:
 test_xgboost(dtsm)

acc1: 0.6177891446433645
prec1: 0.8525951733679886
recall1: 0.9413858868404323
fscore1: 0.8890975900058737
acc_ex: 0.536
prec_ex: 0.9346506242957994
rec_ex: 0.7863182167563413
f1_ex: 0.8401503538252287


In [144]:
from sklearn.neighbors import KNeighborsClassifier
def test_knnm(dtsm):
    knnm=KNeighborsClassifier()
    clsm=MultiOutputClassifier(knnm)
    clsm.fit(dtsm[0],dtsm[2])
    #y_predict=clsm.fit(dtsm[0],dtsm[1]).predict(dtsm[2])
    y_predict=clsm.predict(dtsm[1])    
    print("acc1:",metrics.accuracy_score(y_predict,dtsm[3]))
    print("prec1:",metrics.precision_score(y_predict,dtsm[3], average='weighted'))
    print("recall1:",metrics.recall_score(y_predict,dtsm[3], average='weighted'))
    print("fscore1:",metrics.f1_score(y_predict,dtsm[3], average='weighted'))
    
    y_pred_ex=clsm.predict(x_ex)
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted')) 

In [145]:
test_knnm(dtsm)

acc1: 0.848820687695368
prec1: 0.9393223765071415
recall1: 0.9503945885005637
fscore1: 0.9440044228095952
acc_ex: 0.906
prec_ex: 0.970392710992037
rec_ex: 0.9515757109915449
f1_ex: 0.9600883341205763
